In [38]:
"""
Q3: Extra Work
"""
import keras
from keras import layers
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.datasets import imdb
import torch
import torch.nn as nn
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from nltk.probability import FreqDist

In [85]:
def standardize(x):
    return x.lower()

def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

# prepare data
all_categories = ['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X']
n_classes = len(all_categories)

tagged_sentences = nltk.corpus.treebank.tagged_sents(tagset='universal')
all_sentences = []
for sentence in tagged_sentences:
    t = [standardize(word) for (word, tag) in sentence]
    all_sentences.append(t)

# convert words to indices
word_fdist = FreqDist(
    word for word in flatten(all_sentences)
)
sorted_words = [word for (word, freq) in word_fdist.most_common(n=None)]
word_indices = {word: idx for (idx, word) in enumerate(sorted_words)}
n_words = len(word_indices)

# vectorize data
X = [[word_indices[word] for word in s] for s in all_sentences]
y = [[all_categories.index(tag) for (word, tag) in sentence] for sentence in tagged_sentences]

# pad sequences
sentence_length = 200
X = sequence.pad_sequences(X, maxlen=sentence_length)
y = sequence.pad_sequences(y, maxlen=sentence_length)

In [92]:
"""
Run k-fold cross-validation & evaluate models.
"""
max_features = n_words

def run_cv(model_idx, cv):
    # evaluate models
    results = []
    k_fold = KFold(n_splits=5, shuffle=True, random_state=777)
    for k, (train_idx, test_idx) in enumerate(k_fold.split(X, y)):

        # create model
        model = Sequential()
        embeddings = Embedding(
            input_dim=n_words,
            output_dim=cv['embedding_dim'],
        )
        model.add(embeddings)
        model.add(LSTM(cv['lstm_nodes'], return_sequences=True))
        model.add(Dense(n_classes)) # , activation='softmax'
        model.compile(
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=cv['optimizer'],
            metrics=['accuracy']
        )
        # print(model.summary())
        
        # split data
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]

        # train the model
        model.fit(
            X_train,
            y_train, # y_train[:, -1],
            # validation_data=(X_test, y_test),
            epochs=3,
            batch_size=64
        )

        # final evaluation of the model
        scores = model.evaluate(
            X_test,
            y_test, # y_test[:, -1],
            verbose=0
        )
        accuracy = scores[1]
        
        # report results
        print("Accuracy: %.2f%%" % (accuracy * 100))
        results.append({
            'fold': k,
            'accuracy': accuracy,
            'model_id': model_idx,
            'optimizer': cv['optimizer'],
            'embedding_dim': cv['embedding_dim'],
            'lstm_nodes': cv['lstm_nodes'],
        })
    
    # return results
    return results

In [93]:
# define cross-validation search parameters
cv_params = {
    'optimizer': ['rmsprop', 'adam'],
    'embedding_dim': [32, 64, 128],
    'lstm_nodes': [25, 50, 100],
}

# set the random seed
torch.manual_seed(777)

# run & evaluate each model configuration
results = []
for model_idx, cv in enumerate(list(ParameterGrid(cv_params))):
    # debug
    print(f"# model_idx={model_idx}, {cv}\n")
    
    # run cross-validation & report results
    results.append(
        run_cv(
            model_idx=model_idx,
            cv=cv,
        )
    )

# model_idx=0, {'embedding_dim': 32, 'lstm_nodes': 25, 'optimizer': 'rmsprop'}

Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.8054 - loss: 1.1516
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9041 - loss: 0.3514
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - accuracy: 0.9074 - loss: 0.2959
Accuracy: 91.31%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.8077 - loss: 1.1443
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.9031 - loss: 0.3442
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9088 - loss: 0.2872
Accuracy: 90.95%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.8092 - loss: 0.9485
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.9038 - loss: 0.3253
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.9081 - loss: 0.2851
Accuracy: 91.10%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.8814 - loss: 0.8066
Epoch 2/3
49/49 ━━━━━━━━━━━━━

49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - accuracy: 0.8093 - loss: 0.7398
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.9086 - loss: 0.2887
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.9129 - loss: 0.2704
Accuracy: 92.04%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 128ms/step - accuracy: 0.8119 - loss: 0.7414
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.9094 - loss: 0.2870
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.9134 - loss: 0.2693
Accuracy: 91.68%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 155ms/step - accuracy: 0.8119 - loss: 0.7751
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - accuracy: 0.9073 - loss: 0.2928
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 175ms/step - accuracy: 0.9137 - loss: 0.2687
Accuracy: 91.44%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 153ms/step - accuracy: 0.8115 - loss: 0.7432
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 153ms/step - accuracy: 0.9091 - loss: 0.2870
Epoch 3/3
49/49 ━━━━━━

49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - accuracy: 0.8090 - loss: 0.7300
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.9071 - loss: 0.2935
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.9135 - loss: 0.2692
Accuracy: 91.31%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.8100 - loss: 0.7211
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - accuracy: 0.9072 - loss: 0.2945
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.9115 - loss: 0.2731
Accuracy: 91.25%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.8111 - loss: 0.7505
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.9080 - loss: 0.2916
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.9130 - loss: 0.2722
Accuracy: 91.49%
# model_idx=9, {'embedding_dim': 64, 'lstm_nodes': 50, 'optimizer': 'adam'}

Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - accuracy: 0.8070 - loss: 0.8986
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 83

49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - accuracy: 0.8098 - loss: 0.7373
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.9066 - loss: 0.3053
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.9105 - loss: 0.2724
Accuracy: 91.39%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.8060 - loss: 0.7452
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.9031 - loss: 0.3195
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.9091 - loss: 0.2824
Accuracy: 91.48%
# model_idx=13, {'embedding_dim': 128, 'lstm_nodes': 25, 'optimizer': 'adam'}

Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - accuracy: 0.8086 - loss: 0.9547
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.9070 - loss: 0.3192
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.9170 - loss: 0.2661
Accuracy: 92.78%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.8100 - loss: 0.8182
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 

49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 162ms/step - accuracy: 0.8141 - loss: 0.6319
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - accuracy: 0.9091 - loss: 0.2860
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.9141 - loss: 0.2652
Accuracy: 92.15%
# model_idx=17, {'embedding_dim': 128, 'lstm_nodes': 100, 'optimizer': 'adam'}

Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 159ms/step - accuracy: 0.8114 - loss: 0.7037
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - accuracy: 0.9131 - loss: 0.2628
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 156ms/step - accuracy: 0.9396 - loss: 0.1978
Accuracy: 96.30%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 12s 184ms/step - accuracy: 0.8900 - loss: 0.7268
Epoch 2/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 182ms/step - accuracy: 0.9170 - loss: 0.2561
Epoch 3/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 181ms/step - accuracy: 0.9402 - loss: 0.1927
Accuracy: 96.44%
Epoch 1/3
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 177ms/step - accuracy: 0.8116 - loss: 0.7344
Epoch 2/3
49/49 ━━━━━━━━━

In [94]:
# analyze run results
df = pd.DataFrame.from_records(flatten(results)).set_index(['model_id', 'fold'])
print(df)
agg_metrics = df.groupby(by=['model_id'])['accuracy'].agg("mean")
print(agg_metrics)

               accuracy optimizer  embedding_dim  lstm_nodes
model_id fold                                               
0        0     0.913116   rmsprop             32          25
         1     0.909470   rmsprop             32          25
         2     0.910983   rmsprop             32          25
         3     0.912024   rmsprop             32          25
         4     0.913664   rmsprop             32          25
...                 ...       ...            ...         ...
17       0     0.962982      adam            128         100
         1     0.964381      adam            128         100
         2     0.967740      adam            128         100
         3     0.962937      adam            128         100
         4     0.965154      adam            128         100

[90 rows x 4 columns]
model_id
0     0.911852
1     0.913142
2     0.913638
3     0.919325
4     0.918806
5     0.922802
6     0.913018
7     0.920977
8     0.914553
9     0.929244
10    0.920149
11    0.94